In [9]:
# Packages installation
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from textblob import TextBlob
import seaborn as sns
import nltk
nltk.download('stopwords')


[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


False

In [10]:
# Data Importation
Dataset = pd.read_excel("/Users/adamdanielgreen/Desktop/Business Statistics/Text Analytics-20231219/Data_test_analytics.xlsx")
print(Dataset.head())

                                               Texts          Authors
0  Success in creating AI would be the biggest ev...  Stephen Hawking
1  If our era is the next Industrial Revolution, ...       Fei-Fei Li
2  Elon Musk is worried about AI apocalypse, but ...        Andrew Ng
3  We can build a much brighter future where huma...        Andrew Ng
4  Deep-learning will transform every single indu...        Andrew Ng


In [11]:
# Data subsetting
Comments = Dataset['Texts']

# Remove potential emojis
Comments = Comments.str.encode('ascii', 'ignore').str.decode('ascii')

# All Data are put into a corpus (Collection of comments)
Comments_corpus = Comments.tolist()

# Data are converted to lower case
Comments_corpus = [comment.lower() for comment in Comments_corpus]

# We remove punctuation
Comments_corpus = [re.sub(r'[^\w\s]', '', comment) for comment in Comments_corpus]

# We remove numbers
Comments_corpus = [re.sub(r'\d+', '', comment) for comment in Comments_corpus]

In [16]:
import nltk
nltk.download('stopwords')


[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


False

In [17]:
# We remove non-necessary terms: stopwords
stop_words = set(stopwords.words("english"))
Comments_corpus = [' '.join([word for word in comment.split() if word not in stop_words]) for comment in Comments_corpus]

# We remove our own stopwords
custom_stopwords = ["make", "every", "going", "many", "will", "now", "can", "one", "want", "way", "just"]
Comments_corpus = [' '.join([word for word in comment.split() if word not in custom_stopwords]) for comment in Comments_corpus]

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/adamdanielgreen/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
# The document-term matrix
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
data_final_tdm = vectorizer.fit_transform(Comments_corpus)
m = data_final_tdm.toarray()

In [ ]:
# Display the first 6 rows and all columns of the matrix 'm'
print(m[:6, :])

In [ ]:
# Most frequent terms in our matrix
sums = m.sum(axis=0)
words = vectorizer.get_feature_names_out()
word_freq = dict(zip(words, sums))
sorted_word_freq = dict(sorted(word_freq.items(), key=lambda x: x[1], reverse=True))

# Display the top 10 words and their frequencies
print(list(sorted_word_freq.items())[:10])

# Word Frequency
word_freq_df = pd.DataFrame(list(sorted_word_freq.items())[:15], columns=['word', 'freq'])
plt.figure(figsize=(10, 6))
sns.barplot(x="word", y="freq", data=word_freq_df)
plt.xticks(rotation=45, ha='right')
plt.title("Most frequent words")
plt.xlabel("Word")
plt.ylabel("Word frequencies")
plt.show()

In [ ]:
from wordcloud import WordCloud

# Generate a word cloud image based on word frequencies
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(sorted_word_freq)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Word Cloud of Most Frequent Words")
plt.show()

In [ ]:
# Sentiment Analysis: Global sentiment
from textblob import TextBlob

Comments_bloc = " ".join(Comments_corpus)
text_blob = TextBlob(Comments_bloc)
global_sentiment = text_blob.sentiment
print(global_sentiment)

In [ ]:
# Calculate sentiment scores for each comment
sentiments = [TextBlob(comment).sentiment for comment in Comments_corpus]

# Calculate the average sentiment
average_sentiment = np.mean(sentiments)

# Calculate the standard deviation
std_deviation = np.std(sentiments)

# Print the results
print("Average Sentiment:", average_sentiment)
print("Standard Deviation:", std_deviation)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the NRC Emotion Lexicon from an Excel file
nrc_lexicon = pd.read_excel("/Users/adamdanielgreen/Desktop/Business Statistics/Text Analytics-20231219/NRC-Emotion-Lexicon.xlsx")

# Define the emotion categories you want to analyze
emotion_categories = ["Anger", "Anticipation", "Disgust", "Fear", "Joy", "Negative", "Positive", "Sadness", "Surprise", "Trust"]

# Create a dictionary to store the counts of each emotion category
emotion_counts = {emotion: [] for emotion in emotion_categories}

# Tokenize comments into words
tokenized_comments = [comment.split() for comment in Comments_corpus]

# Calculate emotion counts for each comment
for comment in tokenized_comments:
    for emotion in emotion_categories:
        emotion_count = nrc_lexicon[nrc_lexicon[emotion] == 1]['English (en)'].isin(comment).sum()
        emotion_counts[emotion].append(emotion_count)

# Create a DataFrame from the emotion counts
emotion_df = pd.DataFrame(emotion_counts)

# Display the DataFrame
print(emotion_df.head())

In [ ]:
# Plot the emotion counts
plt.figure(figsize=(12, 6))
for emotion in emotion_categories:
    plt.bar(emotion, emotion_df[emotion].mean(), color='skyblue')

plt.xlabel("Emotion Category")
plt.ylabel("Average Emotion Count")
plt.title("Average Emotion Counts for Emotion Categories")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Plot the sentiment scores
plt.figure(figsize=(12, 6))
sentiment_types = list(sentiment_df.columns)
for sentiment_type in sentiment_types:
    plt.bar(sentiment_type, sentiment_df[sentiment_type].mean())